In [1]:
import pandas as pd
import numpy as np
from rouge_metric import PyRouge
# from rouge import Rouge
from sacrebleu.metrics import BLEU
from functools import reduce 

In [2]:
expert1_file = '/cronus_data/avirinchipur/reasoning_for_psych/expts/parsed_responses/Item_level_scores_Katarina2.csv'
expert2_file = '/cronus_data/avirinchipur/reasoning_for_psych/expts/parsed_responses/Item_level_scores_Veerle3.csv'
self_report_file = '/cronus_data/avirinchipur/reasoning_for_psych/expts/parsed_responses/self_report_unified.csv'
gpt4_file = '/cronus_data/avirinchipur/reasoning_for_psych/expts/parsed_responses/expt_gpt-4-1106-preview.dep_list_phq9items_score_classify2_editted_unified.csv'

In [3]:
SPAN_COLUMNS = ['spans_Anhedonia', 'spans_Depressed_Mood',
       'spans_Insomnia_or_Hypersomnia', 'spans_Fatigue',
       'spans_Poor_appetite_or_overeating', 'spans_Worthlessness_or_Guilt',
       'spans_Difficulty_concentrating',
       'spans_Psychomotor_agitation_or_retardation',
       'spans_Suicidal_ideation']
bleu_scorer = BLEU()
rouge = PyRouge(rouge_n=(1, 2, 3, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

In [4]:
expert1_df = pd.read_csv(expert1_file)
expert2_df = pd.read_csv(expert2_file)
self_report_df = pd.read_csv(self_report_file)
gpt4_df = pd.read_csv(gpt4_file)

In [5]:
print (expert1_df.shape, expert2_df.shape)
print (len(set(expert1_df.user_id).intersection(set(expert2_df.user_id))))

(209, 20) (209, 20)
209


In [6]:
self_report_df.columns

Index(['user_id', 'user_text', 'score_Anhedonia', 'score_Depressed_Mood',
       'score_Insomnia_or_Hypersomnia', 'score_Fatigue',
       'score_Poor_appetite_or_overeating', 'score_Worthlessness_or_Guilt',
       'score_Difficulty_concentrating',
       'score_Psychomotor_agitation_or_retardation',
       'score_Suicidal_ideation'],
      dtype='object')

In [7]:
gpt4_df.columns

Index(['user_id', 'user_text', 'score_Anhedonia', 'score_Depressed_Mood',
       'score_Insomnia_or_Hypersomnia', 'score_Fatigue',
       'score_Poor_appetite_or_overeating', 'score_Worthlessness_or_Guilt',
       'score_Difficulty_concentrating',
       'score_Psychomotor_agitation_or_retardation', 'score_Suicidal_ideation',
       'spans_Anhedonia', 'spans_Depressed_Mood',
       'spans_Insomnia_or_Hypersomnia', 'spans_Fatigue',
       'spans_Poor_appetite_or_overeating', 'spans_Worthlessness_or_Guilt',
       'spans_Difficulty_concentrating',
       'spans_Psychomotor_agitation_or_retardation', 'spans_Suicidal_ideation',
       'isInferred_Anhedonia', 'isInferred_Depressed_Mood',
       'isInferred_Insomnia_or_Hypersomnia', 'isInferred_Fatigue',
       'isInferred_Poor_appetite_or_overeating',
       'isInferred_Worthlessness_or_Guilt',
       'isInferred_Difficulty_concentrating',
       'isInferred_Psychomotor_agitation_or_retardation',
       'isInferred_Suicidal_ideation'],
    

In [8]:
print (expert1_df.columns)
expert1_df.head()

Index(['user_id', 'user_text', 'score_Anhedonia', 'score_Depressed_Mood',
       'score_Insomnia_or_Hypersomnia', 'score_Fatigue',
       'score_Poor_appetite_or_overeating', 'score_Worthlessness_or_Guilt',
       'score_Difficulty_concentrating',
       'score_Psychomotor_agitation_or_retardation', 'score_Suicidal_ideation',
       'spans_Anhedonia', 'spans_Depressed_Mood',
       'spans_Insomnia_or_Hypersomnia', 'spans_Fatigue',
       'spans_Poor_appetite_or_overeating', 'spans_Worthlessness_or_Guilt',
       'spans_Difficulty_concentrating',
       'spans_Psychomotor_agitation_or_retardation',
       'spans_Suicidal_ideation'],
      dtype='object')


,user_id,user_text,score_Anhedonia,score_Depressed_Mood,score_Insomnia_or_Hypersomnia,score_Fatigue,score_Poor_appetite_or_overeating,score_Worthlessness_or_Guilt,score_Difficulty_concentrating,score_Psychomotor_agitation_or_retardation,score_Suicidal_ideation,spans_Anhedonia,spans_Depressed_Mood,spans_Insomnia_or_Hypersomnia,spans_Fatigue,spans_Poor_appetite_or_overeating,spans_Worthlessness_or_Guilt,spans_Difficulty_concentrating,spans_Psychomotor_agitation_or_retardation,spans_Suicidal_ideation
0,45,My depression is something I've learnt to live...,NaN,3,1.0,1.0,2.0,2.0,2,0.0,0.0,[],"[""My depression is something I've learnt to li...",[],[],[],[],[],[],[]
1,49,It’s becoming really stressful at the moment o...,1.0,2,0.0,1.0,0.0,0.0,0,0.0,0.0,"['It’s becoming really stressful', 'we are not...","['It’s becoming really stressful', 'we are not...",[],[],[],[],[],[],[]
2,70,Yes I am regularly depressed and end up drinki...,3.0,3,2.0,0.0,2.0,2.0,2,0.0,1.0,"['I am ok to around lunchtime, but then there ...",['I am regularly depressed and end up drinking...,[],[],[],[],[],[],[]
3,71,Over the last two weeks my mood has been posit...,0.0,1,0.0,0.0,0.0,1.0,0,0.0,0.0,"['Life feels like it is improving', 'I am able...","['my mood has been positiv', 'I have not felt ...",[],[],[],[],[],[],[]
4,72,I'd been a little depressed off and on. Mainly...,2.0,2,1.0,1.0,2.0,1.0,1,0.0,0.0,"[""I've been stuck at home for the most part"", ...","['little depressed off and on.', 'rather lonel...",[],[],[],[],[],[],[]


In [9]:
## DO NOT USE THIS CODE. THIS CELL CONTAINS OLD ROUGE CALCULATIONS CODE> DO NOT USE

# expert1_gold_r1, expert1_gold_r2, expert1_gold_rl = {}, {}, {}
# expert2_gold_r1, expert2_gold_r2, expert2_gold_rl = {}, {}, {}
# for col in SPAN_COLUMNS:
#     temp = pd.merge(expert1_df[['user_id', col]], expert2_df[['user_id', col]], on='user_id', suffixes=('_expert1', '_expert2'))
#     temp = temp[~((temp['{}_expert1'.format(col)] == '[]') & (temp['{}_expert2'.format(col)] == '[]'))]
#     temp1 = temp['{}_expert1'.format(col)].tolist()
#     temp2 = temp['{}_expert2'.format(col)].tolist()
#     expert1_gold_rouge = rouge_scorer.get_scores(temp1, temp2, avg=True, ignore_empty=False)
#     expert2_gold_rouge = rouge_scorer.get_scores(temp2, temp1, avg=True, ignore_empty=False)
#     expert1_gold_r1[col] = [expert1_gold_rouge['rouge-1']['f'],] if col not in expert1_gold_r1 else expert1_gold_r1[col] + [expert1_gold_rouge['rouge-1']['f'],]
#     expert2_gold_r1[col] = [expert2_gold_rouge['rouge-1']['f'],] if col not in expert2_gold_r1 else expert2_gold_r1[col] + [expert2_gold_rouge['rouge-1']['f'],]
#     expert1_gold_r2[col] = [expert1_gold_rouge['rouge-2']['f'],] if col not in expert1_gold_r2 else expert1_gold_r2[col] + [expert1_gold_rouge['rouge-2']['f'],]
#     expert2_gold_r2[col] = [expert2_gold_rouge['rouge-2']['f'],] if col not in expert2_gold_r2 else expert2_gold_r2[col] + [expert2_gold_rouge['rouge-2']['f'],]
#     expert1_gold_rl[col] = [expert1_gold_rouge['rouge-l']['f'],] if col not in expert1_gold_rl else expert1_gold_rl[col] + [expert1_gold_rouge['rouge-l']['f'],]
#     expert2_gold_rl[col] = [expert2_gold_rouge['rouge-l']['f'],] if col not in expert2_gold_rl else expert2_gold_rl[col] + [expert2_gold_rouge['rouge-l']['f'],]

In [10]:
# e1e2_r1 = pd.concat([pd.DataFrame(expert1_gold_r1).T.rename(columns={0:'rouge-1-e1-gold'}), pd.DataFrame(expert2_gold_r1).T.rename(columns={0:'rouge-1-e2-gold'})], axis=1)
# e1e2_r2 = pd.concat([pd.DataFrame(expert1_gold_r2).T.rename(columns={0:'rouge-2-e1-gold'}), pd.DataFrame(expert2_gold_r2).T.rename(columns={0:'rouge-2-e2-gold'})], axis=1)
# e1e2_rl = pd.concat([pd.DataFrame(expert1_gold_rl).T.rename(columns={0:'rouge-l-e1-gold'}), pd.DataFrame(expert2_gold_rl).T.rename(columns={0:'rouge-l-e2-gold'})], axis=1)
# e1e2_rouge = pd.concat([e1e2_r1, e1e2_r2, e1e2_rl], axis=1)
# e1e2_rouge    

In [11]:
expert1_gold_bleu = {}
expert2_gold_bleu = {}
gpt4_bleu_scores_e1 = {}
gpt4_bleu_scores_e2 = {}
gpt4_bleu_scores_e1e2 = {}

for col in SPAN_COLUMNS:
    temp = pd.merge(expert1_df[['user_id', col]], expert2_df[['user_id', col]], on='user_id', suffixes=('_expert1', '_expert2'))
    temp = temp[~((temp['{}_expert1'.format(col)] == '[]') & (temp['{}_expert2'.format(col)] == '[]'))]
    temp1 = temp['{}_expert1'.format(col)].tolist()
    temp2 = temp['{}_expert2'.format(col)].tolist()
    temp1 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp1]
    temp2 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp2]
    max_num_spans_t1 = max([len(t) for t in temp1])
    max_num_spans_t2 = max([len(t) for t in temp2])
    
    refs_t1, refs_t2 = [['']*len(temp1) for j in range(max_num_spans_t1)], [['']*len(temp2) for j in range(max_num_spans_t2)] 
    for idx, row in enumerate(temp1):
        for j, span in enumerate(row):
            refs_t1[j][idx] = span
    for idx, row in enumerate(temp2):
        for j, span in enumerate(row):
            refs_t2[j][idx] = span
    
    if col not in expert1_gold_bleu: expert1_gold_bleu[col] = []
    if col not in expert2_gold_bleu: expert2_gold_bleu[col] = []
    if col not in gpt4_bleu_scores_e1: gpt4_bleu_scores_e1[col] = []
    if col not in gpt4_bleu_scores_e2: gpt4_bleu_scores_e2[col] = []
    if col not in gpt4_bleu_scores_e1e2: gpt4_bleu_scores_e1e2[col] = []

    for idx in range(temp.shape[0]):
        current_user_id = temp.user_id.iloc[idx].item()
        current_users_score = -float('inf')
        ref = [[refs_t1[i][idx]] for i in range(len(refs_t1))]
        if not all([span == [''] for span in ref]):
            for jdx in range(max_num_spans_t2):     
                sys = [refs_t2[jdx][idx]]
                current_score = bleu_scorer.corpus_score(references=ref, hypotheses=sys).score
                current_users_score = max(current_score, current_users_score)
            expert1_gold_bleu[col].append((current_user_id, current_users_score))
            
        current_users_score = -float('inf')
        ref = [[refs_t2[i][idx]] for i in range(len(refs_t2))]
        if not all([span == [''] for span in ref]):
            for kdx in range(max_num_spans_t1):
                sys = [refs_t1[kdx][idx]]
                current_score = bleu_scorer.corpus_score(references=ref, hypotheses=sys).score
                current_users_score = max(current_score, current_users_score)
            expert2_gold_bleu[col].append((current_user_id, current_users_score))

    gpt4_filter_df = gpt4_df[gpt4_df.user_id.isin(temp.user_id.tolist())]
    gpt4_spans = gpt4_filter_df[col].tolist()
    for idx in range(gpt4_filter_df.shape[0]):
        if gpt4_spans[idx].startswith('['):
            gpt4_spans[idx] = eval(gpt4_spans[idx])
        else:
            gpt4_spans[idx] = ['']
        sys = gpt4_spans[idx]
        ref_e1 = [[refs_t1[i][idx]] for i in range(len(refs_t1))]
        ref_e2 = [[refs_t2[i][idx]] for i in range(len(refs_t2))]
        bleu_score_e1, bleu_score_e2, bleu_score_e1e2 = -float('inf'), -float('inf'), -float('inf')
        for sys_span in sys:
            if not all([span == [''] for span in ref_e1]): bleu_score_e1 = max(bleu_scorer.corpus_score(references=ref_e1, hypotheses=[sys_span]).score, bleu_score_e1)
            if not all([span == [''] for span in ref_e2]): bleu_score_e2 = max(bleu_scorer.corpus_score(references=ref_e2, hypotheses=[sys_span]).score, bleu_score_e2)
            if not all([span == [''] for span in ref_e1+ref_e2]): bleu_score_e1e2 = max(bleu_scorer.corpus_score(references=ref_e1+ref_e2, hypotheses=[sys_span]).score, bleu_score_e1e2)
        if bleu_score_e1 != -float('inf'): gpt4_bleu_scores_e1[col].append((gpt4_filter_df['user_id'].iloc[idx], bleu_score_e1))
        if bleu_score_e2 != -float('inf'): gpt4_bleu_scores_e2[col].append((gpt4_filter_df['user_id'].iloc[idx], bleu_score_e2))
        if bleu_score_e1e2 != -float('inf'): gpt4_bleu_scores_e1e2[col].append((gpt4_filter_df['user_id'].iloc[idx], bleu_score_e1e2))

In [12]:
pd.DataFrame(gpt4_bleu_scores_e1[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()

0.0

In [13]:
bleu_scores = {}
for col in SPAN_COLUMNS:
    expert1_gold_bleu_score = pd.DataFrame(expert1_gold_bleu[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    expert2_gold_bleu_score = pd.DataFrame(expert2_gold_bleu[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e1 = pd.DataFrame(gpt4_bleu_scores_e1[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e2 = pd.DataFrame(gpt4_bleu_scores_e2[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e1e2 = pd.DataFrame(gpt4_bleu_scores_e1e2[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    bleu_scores[col] = (expert1_gold_bleu_score, expert2_gold_bleu_score, gpt4_bleu_score_e1, gpt4_bleu_score_e2, gpt4_bleu_score_e1e2)
pd.DataFrame(bleu_scores).T.rename(columns={0:'bleu-e1-gold', 1:'bleu-e2-gold', 2:'bleu-gpt4-e1', 3:'bleu-gpt4-e2', 4:'bleu-gpt4-e1e2'})    


,bleu-e1-gold,bleu-e2-gold,bleu-gpt4-e1,bleu-gpt4-e2,bleu-gpt4-e1e2
spans_Anhedonia,65.431173,45.632725,3.698496,3.476460,3.623141
spans_Depressed_Mood,83.233170,65.601646,9.550970,10.340225,10.855234
spans_Insomnia_or_Hypersomnia,58.488036,51.417717,2.267885,3.399842,3.294241
spans_Fatigue,25.405090,23.282012,1.911097,1.632418,2.056134
spans_Poor_appetite_or_overeating,72.434806,47.121316,10.697173,8.307986,8.307986
spans_Worthlessness_or_Guilt,64.801982,24.312320,3.946702,2.073047,2.482946
spans_Difficulty_concentrating,42.101295,9.474970,0.961208,1.528425,1.433146
spans_Psychomotor_agitation_or_retardation,50.000000,33.333333,1.693249,1.128833,0.846625
spans_Suicidal_ideation,66.445874,6.181637,0.000000,0.883176,0.883176


In [14]:
gpt4_bleu_scores_e1e2['spans_Psychomotor_agitation_or_retardation']

[(188, 0.0), (193, 0.0), (499, 3.3864985683445354), (832, 0.0)]

In [15]:
bleu_scores = {}
for col in SPAN_COLUMNS:
    expert1_gold_bleu_score = pd.DataFrame(expert1_gold_bleu[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    expert2_gold_bleu_score = pd.DataFrame(expert2_gold_bleu[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e1 = pd.DataFrame(gpt4_bleu_scores_e1[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e2 = pd.DataFrame(gpt4_bleu_scores_e2[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    gpt4_bleu_score_e1e2 = pd.DataFrame(gpt4_bleu_scores_e1e2[col], columns=['user_id', 'bleu_score'])['bleu_score'].mean()
    bleu_scores[col] = (expert1_gold_bleu_score, expert2_gold_bleu_score, gpt4_bleu_score_e1, gpt4_bleu_score_e2, gpt4_bleu_score_e1e2)
pd.DataFrame(bleu_scores).T.rename(columns={0:'bleu-e1-gold', 1:'bleu-e2-gold', 2:'bleu-gpt4-e1', 3:'bleu-gpt4-e2', 4:'bleu-gpt4-e1e2'})    


,bleu-e1-gold,bleu-e2-gold,bleu-gpt4-e1,bleu-gpt4-e2,bleu-gpt4-e1e2
spans_Anhedonia,65.431173,45.632725,3.698496,3.476460,3.623141
spans_Depressed_Mood,83.233170,65.601646,9.550970,10.340225,10.855234
spans_Insomnia_or_Hypersomnia,58.488036,51.417717,2.267885,3.399842,3.294241
spans_Fatigue,25.405090,23.282012,1.911097,1.632418,2.056134
spans_Poor_appetite_or_overeating,72.434806,47.121316,10.697173,8.307986,8.307986
spans_Worthlessness_or_Guilt,64.801982,24.312320,3.946702,2.073047,2.482946
spans_Difficulty_concentrating,42.101295,9.474970,0.961208,1.528425,1.433146
spans_Psychomotor_agitation_or_retardation,50.000000,33.333333,1.693249,1.128833,0.846625
spans_Suicidal_ideation,66.445874,6.181637,0.000000,0.883176,0.883176


In [16]:
expert1_gold_rouge = {}
expert2_gold_rouge = {}
gpt4_rouge_scores_e1 = {}
gpt4_rouge_scores_e2 = {}
gpt4_rouge_scores_e1e2 = {}
metric = "p"


for col in SPAN_COLUMNS:
    temp = pd.merge(expert1_df[['user_id', col]], expert2_df[['user_id', col]], on='user_id', suffixes=('_expert1', '_expert2'))
    temp = temp[~((temp['{}_expert1'.format(col)] == '[]') & (temp['{}_expert2'.format(col)] == '[]'))]
    temp1 = temp['{}_expert1'.format(col)].tolist()
    temp2 = temp['{}_expert2'.format(col)].tolist()
    temp1 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp1]
    temp2 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp2]
    max_num_spans_t1 = max([len(t) for t in temp1])
    max_num_spans_t2 = max([len(t) for t in temp2])
    
    if col not in expert1_gold_rouge: expert1_gold_rouge[col] = []
    if col not in expert2_gold_rouge: expert2_gold_rouge[col] = []
    if col not in gpt4_rouge_scores_e1: gpt4_rouge_scores_e1[col] = []
    if col not in gpt4_rouge_scores_e2: gpt4_rouge_scores_e2[col] = []
    if col not in gpt4_rouge_scores_e1e2: gpt4_rouge_scores_e1e2[col] = []


    for idx in range(temp.shape[0]):
        current_user_id = temp.user_id.iloc[idx].item()
        current_users_score = -float('inf')
        references = [temp1[idx]]
        for jdx in range(len(temp2[idx])):
            hypotheses = [temp2[idx][jdx]]
            current_score = reduce(max, [rouge.evaluate(hypotheses, [[reference]])['rouge-l'][metric] for reference in references[0]])
            current_users_score = max(current_score, current_users_score)
        expert1_gold_rouge[col].append((current_user_id, current_users_score))

        current_users_score = -float('inf')
        references = [temp2[idx]]
        for kdx in range(len(temp1[idx])):    
            hypotheses = [temp1[idx][kdx]]
            current_score = reduce(max, [rouge.evaluate(hypotheses, [[reference]])['rouge-l'][metric] for reference in references[0]])
            current_users_score = max(current_score, current_users_score)
        expert2_gold_rouge[col].append((current_user_id, current_users_score))

    gpt4_filter_df = pd.merge(gpt4_df[['user_id', col]], temp, on='user_id', suffixes=('', '_experts'))
    gpt4_spans = gpt4_filter_df[col].tolist()
    temp1 = gpt4_filter_df['{}_expert1'.format(col)].tolist()
    temp2 = gpt4_filter_df['{}_expert2'.format(col)].tolist()
    temp1 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp1]
    temp2 = [eval(t) if len(eval(t)) > 0 else [''] for t in temp2]
        
    for idx in range(gpt4_filter_df.shape[0]):
        try:
            gpt4_spans[idx] = eval(gpt4_spans[idx])
        except:
            gpt4_spans[idx] = ['']
        sys = gpt4_spans[idx]
        ref_e1 = [temp1[idx]]
        ref_e2 = [temp2[idx]]
        ref_e1e2 = [temp1[idx]+temp2[idx]]

        rouge_score_e1, rouge_score_e2, rouge_score_e1e2 = -float('inf'), -float('inf'), -float('inf')
        for sys_span in sys:
            if not all([span == [''] for span in ref_e1]) or True: 
                current_score = reduce(max, [rouge.evaluate([sys_span], [[reference]])['rouge-l'][metric] for reference in ref_e1[0]])
                rouge_score_e1 = max(current_score, rouge_score_e1)
            if not all([span == [''] for span in ref_e2]) or True: 
                current_score = reduce(max, [rouge.evaluate([sys_span], [[reference]])['rouge-l'][metric] for reference in ref_e2[0]])
                rouge_score_e2 = max(current_score, rouge_score_e2)
            if not all([span == [''] for span in ref_e1+ref_e2]) or True:
                current_score = reduce(max, [rouge.evaluate([sys_span], [[reference]])['rouge-l'][metric] for reference in ref_e1[0]+ref_e2[0]])
                rouge_score_e1e2 = max(current_score, rouge_score_e1e2)
        if rouge_score_e1 != -float('inf'): gpt4_rouge_scores_e1[col].append((gpt4_filter_df['user_id'].iloc[idx], rouge_score_e1))
        if rouge_score_e2 != -float('inf'): gpt4_rouge_scores_e2[col].append((gpt4_filter_df['user_id'].iloc[idx], rouge_score_e2))
        if rouge_score_e1e2 != -float('inf'): gpt4_rouge_scores_e1e2[col].append((gpt4_filter_df['user_id'].iloc[idx], rouge_score_e1e2))

In [17]:
expert1_gold_rouge == expert2_gold_rouge

False

In [18]:
rouge_scores = {}
for col in SPAN_COLUMNS:
    expert1_gold_rouge_score = pd.DataFrame(expert1_gold_rouge[col], columns=['user_id', 'rouge_score'])['rouge_score'].mean()
    expert2_gold_rouge_score = pd.DataFrame(expert2_gold_rouge[col], columns=['user_id', 'rouge_score'])['rouge_score'].mean()
    gpt4_rouge_score_e1 = pd.DataFrame(gpt4_rouge_scores_e1[col], columns=['user_id', 'rouge_score'])['rouge_score'].mean()
    gpt4_rouge_score_e2 = pd.DataFrame(gpt4_rouge_scores_e2[col], columns=['user_id', 'rouge_score'])['rouge_score'].mean()
    gpt4_rouge_score_e1e2 = pd.DataFrame(gpt4_rouge_scores_e1e2[col], columns=['user_id', 'rouge_score'])['rouge_score'].mean()
    rouge_scores[col] = (expert1_gold_rouge_score, expert2_gold_rouge_score, gpt4_rouge_score_e1, gpt4_rouge_score_e2, gpt4_rouge_score_e1e2)
rouge_scores_df = pd.DataFrame(rouge_scores).T.rename(columns={0:'rouge-e1-gold', 1:'rouge-e2-gold', 2:'rouge-gpt4-e1', 3:'rouge-gpt4-e2', 4:'rouge-gpt4-e1e2'})
rouge_scores_df

,rouge-e1-gold,rouge-e2-gold,rouge-gpt4-e1,rouge-gpt4-e2,rouge-gpt4-e1e2
spans_Anhedonia,0.472064,0.443490,0.360932,0.475306,0.521525
spans_Depressed_Mood,0.779668,0.691278,0.686557,0.832973,0.879016
spans_Insomnia_or_Hypersomnia,0.527019,0.569372,0.539140,0.694357,0.715106
spans_Fatigue,0.205964,0.291257,0.361140,0.485018,0.603520
spans_Poor_appetite_or_overeating,0.590484,0.624242,0.620238,0.822024,0.822024
spans_Worthlessness_or_Guilt,0.253739,0.246407,0.225338,0.469719,0.478681
spans_Difficulty_concentrating,0.105395,0.093022,0.126068,0.374872,0.386527
spans_Psychomotor_agitation_or_retardation,0.250000,0.250000,0.267857,0.267857,0.392857
spans_Suicidal_ideation,0.068788,0.063847,0.067810,0.223695,0.223695


In [19]:
rouge_scores_df[['rouge-e1-gold', 'rouge-e2-gold' ]].mean(1).round(2)

spans_Anhedonia                               0.46
spans_Depressed_Mood                          0.74
spans_Insomnia_or_Hypersomnia                 0.55
spans_Fatigue                                 0.25
spans_Poor_appetite_or_overeating             0.61
spans_Worthlessness_or_Guilt                  0.25
spans_Difficulty_concentrating                0.10
spans_Psychomotor_agitation_or_retardation    0.25
spans_Suicidal_ideation                       0.07
dtype: float64

In [20]:
# concatenate the spans from both experts in a df
experts_merged_df = expert1_df.copy()
for col in SPAN_COLUMNS:
    temp = pd.merge(expert1_df[['user_id', col]], expert2_df[['user_id', col]], on='user_id', suffixes=('_expert1', '_expert2'))
    experts_merged_df[col] = temp.apply(lambda x: eval(x['{}_expert1'.format(col)]) + eval(x['{}_expert2'.format(col)]), axis=1)

experts_merged_df = experts_merged_df[['user_id', 'user_text'] + SPAN_COLUMNS]
experts_merged_df 

,user_id,user_text,spans_Anhedonia,spans_Depressed_Mood,spans_Insomnia_or_Hypersomnia,spans_Fatigue,spans_Poor_appetite_or_overeating,spans_Worthlessness_or_Guilt,spans_Difficulty_concentrating,spans_Psychomotor_agitation_or_retardation,spans_Suicidal_ideation
0,45,My depression is something I've learnt to live...,[],[My depression is something I've learnt to liv...,[],[],[],[I feel like I'm the only one who feels this w...,[It brings out my anxiety],[],[]
1,49,It’s becoming really stressful at the moment o...,"[It’s becoming really stressful, we are not ha...","[It’s becoming really stressful, we are not ha...",[],[],[],[as a mother of two I think about this],"[It’s becoming really stressful at the moment,...",[],[]
2,70,Yes I am regularly depressed and end up drinki...,"[I am ok to around lunchtime, but then there i...",[I am regularly depressed and end up drinking ...,[],[],[],[],[],[],[]
3,71,Over the last two weeks my mood has been posit...,"[Life feels like it is improving, I am able to...","[my mood has been positiv, I have not felt as ...",[],[],[],[I have been able to reflect upon my life posi...,[],[],[]
4,72,I'd been a little depressed off and on. Mainly...,"[I've been stuck at home for the most part, I ...","[little depressed off and on., rather lonely, ...",[],[],[],[I feel as if I'm getting older and my looks a...,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
204,573,Over the last few weeks I have experienced pro...,[no motivation to complete even simple everyda...,"[I have experienced prolonged depression, I ha...",[],"[I have had no motivation to get up, I haven't...",[],[I've spent a lot of time crying in bed just t...,[],[],[I've spent a lot of time crying in bed just t...
205,575,I have struggled with changes to the lives of ...,[],"[I am racked with guilt and self loathing, a f...",[],[],[],[I constantly see mistakes I have made as nega...,[],[],[]
206,966,I believe I have been depressed over the last ...,[not wanting to get out of bed],[I believe I have been depressed over the last...,"[not wanting to get out of bed, not wanting to...",[],"[I have found myself eating more, I have found...",[],[],[],[]
207,499,I have occasional days or even just moments of...,[over the last few weeks I've just felt kind o...,[I have occasional days or even just moments o...,[],[],[],[],[],[almost floating around unable to move forward],[]


In [21]:
# For each column, Caluculate the number of rows for which experts_merged has at least one span entry and gpt4 doesn't have any 
span_meta_info = []
for col in SPAN_COLUMNS:
    temp = pd.merge(experts_merged_df[['user_id', col]], gpt4_df[['user_id', col]], on='user_id', suffixes=('_experts', '_gpt4'))
    temp['{}_gpt4'.format(col)] = temp['{}_gpt4'.format(col)].apply(lambda x: eval(x) if x.startswith('[') else [])
    num_expert_rows = temp[temp['{}_experts'.format(col)].apply(len) > 0]
    num_gpt4_rows = temp[temp['{}_gpt4'.format(col)].apply(len) > 0]
    num_conditional_rows = temp[(temp['{}_experts'.format(col)].apply(len) > 0) & (temp['{}_gpt4'.format(col)].apply(len) > 0)]
    span_meta_info.append([col, num_expert_rows.shape[0], num_gpt4_rows.shape[0], num_conditional_rows.shape[0]])

span_meta_info = pd.DataFrame(span_meta_info, columns=['column', 'num_expert_rows', 'num_gpt4_rows', 'num_intersection_rows'])
span_meta_info['intersection_rate'] = (span_meta_info['num_intersection_rows']*100 / span_meta_info['num_expert_rows']).round(2)
span_meta_info

,column,num_expert_rows,num_gpt4_rows,num_intersection_rows,intersection_rate
0,spans_Anhedonia,134,118,91,67.91
1,spans_Depressed_Mood,206,202,202,98.06
2,spans_Insomnia_or_Hypersomnia,33,32,26,78.79
3,spans_Fatigue,40,69,31,77.50
4,spans_Poor_appetite_or_overeating,14,18,12,85.71
5,spans_Worthlessness_or_Guilt,87,70,49,56.32
6,spans_Difficulty_concentrating,39,39,17,43.59
7,spans_Psychomotor_agitation_or_retardation,4,32,3,75.00
8,spans_Suicidal_ideation,72,22,18,25.00


In [22]:
rouge_scores_df.reset_index().merge(span_meta_info, left_on="index", right_on='column')

,index,rouge-e1-gold,rouge-e2-gold,rouge-gpt4-e1,rouge-gpt4-e2,rouge-gpt4-e1e2,column,num_expert_rows,num_gpt4_rows,num_intersection_rows,intersection_rate
0,spans_Anhedonia,0.472064,0.443490,0.360932,0.475306,0.521525,spans_Anhedonia,134,118,91,67.91
1,spans_Depressed_Mood,0.779668,0.691278,0.686557,0.832973,0.879016,spans_Depressed_Mood,206,202,202,98.06
2,spans_Insomnia_or_Hypersomnia,0.527019,0.569372,0.539140,0.694357,0.715106,spans_Insomnia_or_Hypersomnia,33,32,26,78.79
3,spans_Fatigue,0.205964,0.291257,0.361140,0.485018,0.603520,spans_Fatigue,40,69,31,77.50
4,spans_Poor_appetite_or_overeating,0.590484,0.624242,0.620238,0.822024,0.822024,spans_Poor_appetite_or_overeating,14,18,12,85.71
5,spans_Worthlessness_or_Guilt,0.253739,0.246407,0.225338,0.469719,0.478681,spans_Worthlessness_or_Guilt,87,70,49,56.32
6,spans_Difficulty_concentrating,0.105395,0.093022,0.126068,0.374872,0.386527,spans_Difficulty_concentrating,39,39,17,43.59
7,spans_Psychomotor_agitation_or_retardation,0.250000,0.250000,0.267857,0.267857,0.392857,spans_Psychomotor_agitation_or_retardation,4,32,3,75.00
8,spans_Suicidal_ideation,0.068788,0.063847,0.067810,0.223695,0.223695,spans_Suicidal_ideation,72,22,18,25.00


In [26]:
col

'spans_Psychomotor_agitation_or_retardation'

In [33]:
col = SPAN_COLUMNS[-1]
temp = pd.merge(experts_merged_df[['user_id', 'user_text', col]], gpt4_df[['user_id', col]], on='user_id', suffixes=('_experts', '_gpt4'))
temp['{}_gpt4'.format(col)] = temp['{}_gpt4'.format(col)].apply(lambda x: eval(x) if x.startswith('[') else [])
temp[(temp['{}_experts'.format(col)].apply(len) > 0) & (temp['{}_gpt4'.format(col)].apply(len) == 0)]

,user_id,user_text,spans_Suicidal_ideation_experts,spans_Suicidal_ideation_gpt4
6,133,I am by nature cheerful and optimistic and ha...,[I am by nature cheerful and optimistic],[]
9,160,I have not been depressed over the last two we...,[I've felt normal and perfectly fine with the ...,[]
11,169,I have not been depressed at all over the last...,[I'm happy with my life and job],[]
12,176,"No I haven't been depressed at all recently, m...",[nothing to affect me mentally],[]
13,188,Luckily I do not suffer depression of any kind...,[Luckily I do not suffer depression of any kind],[]
14,195,I have not been depressed in the past 2 weeks....,[I’ve been trying to keep a more positive outl...,[]
15,241,I have never had an issue with depression so o...,[I have never had an issue with depression],[]
17,316,I'm not sure about medical depression - I've b...,[very different from my first bout of depressi...,[]
22,416,Depression actually sin't a thing for me becau...,[there is more or the least positivity is enou...,[]
26,480,"The last week has been hard, I have not wanted...",[I don't see future],[]


In [30]:
experts_merged_df[experts_merged_df.user_id.isin([549, 251, 571])][['user_text', 'spans_Fatigue']].values

array([['Yes I have been depressed. It has caused me to feel completely exhausted, and I am constantly overthinking. I am high functioning, so still work full time, but once I am home I just want to sleep and feel nothing. I feel empty.',
        list(['feel completely exhausted,', 'I just want to sleep and feel nothing. ', 'It has caused me to feel completely exhausted'])],
       ["At times I have felt extremely lethargic and down. I lost all desire to do anything fun or get dressed I didn't want to see anyone or speak to anyone. I thought there was only one way out. It hits me in waves and then I can feel much better.",
        list([])],
       ["I have felt down over the past 2 weeks, but have tried to ignore it. Mostly, I've felt very lethargic and numb and haven't wanted to get out of bed (unexcited about the day ahead). I know that it is important for me to keep busy so I often try to distract myself and keep myself very busy focusing on work or making plans. I've tried hard to

In [64]:
bleu_scorer_new = BLEU()
predictions = ["hello there general kenobi"]
references = [
     ["hello there general kenobi", "some random text"],
 ]
bleu_scorer_new.corpus_score(references=references, hypotheses=predictions)

rouge.evaluate(predictions, references)

{'rouge-1': {'r': 0.5714285714285714, 'p': 0.5, 'f': 0.5333333333333333},
 'rouge-2': {'r': 0.6, 'p': 0.5, 'f': 0.5454545454545454},
 'rouge-4': {'r': 1.0, 'p': 0.5, 'f': 0.6666666666666666},
 'rouge-l': {'r': 0.5714285714285714, 'p': 0.5, 'f': 0.5333333333333333},
 'rouge-w-1.2': {'r': 0.4965742461602382,
  'p': 0.5612310241546865,
  'f': 0.5269266103360986},
 'rouge-s4': {'r': 0.6666666666666666, 'p': 0.5, 'f': 0.5714285714285715},
 'rouge-su4': {'r': 0.6428571428571429, 'p': 0.5, 'f': 0.5625000000000001}}

In [87]:
bleu_scorer_new = BLEU()
predictions = ["been the most restless"]	
references = [
     ["been the most restless i have ever felt"]
 ]
bleu_scorer_new.corpus_score(references=references, hypotheses=predictions)

rouge.evaluate(predictions, references)

{'rouge-1': {'r': 0.5, 'p': 1.0, 'f': 0.6666666666666666},
 'rouge-2': {'r': 0.42857142857142855, 'p': 1.0, 'f': 0.6},
 'rouge-4': {'r': 0.2, 'p': 1.0, 'f': 0.33333333333333337},
 'rouge-l': {'r': 0.5, 'p': 1.0, 'f': 0.6666666666666666},
 'rouge-w-1.2': {'r': 0.3298769776932236, 'p': 1.0, 'f': 0.49610149393731323},
 'rouge-s4': {'r': 0.24, 'p': 1.0, 'f': 0.3870967741935484},
 'rouge-su4': {'r': 0.28125, 'p': 1.0, 'f': 0.43902439024390244}}

In [22]:
import evaluate
bleu = evaluate.load("bleu")
predictions = ["low appetite"]
references = [
     ["low appetite", "high summer"],
 ]
results = bleu.compute(predictions=predictions, references=references)
print (results)
rouge.evaluate(predictions, [["", ""],])

{'bleu': 0.0, 'precisions': [1.0, 1.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 2, 'reference_length': 2}


{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-w-1.2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

In [51]:
bleu_scorer_new = BLEU()
ref1 = [["low appetite summer"],]
sys1 = ["low appetite summer"]
bleu_scorer_new.corpus_score(references=ref1, hypotheses=sys1)

BLEU = 0.00 100.0/100.0/100.0/0.0 (BP = 1.000 ratio = 1.000 hyp_len = 3 ref_len = 3)

In [44]:
reference = [['the quick brown fox jumped over the lazy dog']]
candidate = ['the fast brown fox jumped over the sleepy dog']
bleu_scorer_new.corpus_score(references=reference, hypotheses=candidate)

BLEU = 48.55 77.8/50.0/42.9/33.3 (BP = 1.000 ratio = 1.000 hyp_len = 9 ref_len = 9)

In [16]:
pd.DataFrame(gpt4_bleu_scores_e1e2['spans_Poor_appetite_or_overeating'], columns=['user_id', 'bleu_score'])

,user_id,bleu_score
0,32,0.000000
1,145,0.000000
2,701,0.000000
3,709,0.000000
4,743,0.000000
5,755,3.817681
6,806,0.000000
7,807,5.522398
8,809,0.000000
9,863,0.000000


In [59]:
temp[temp['{}_gpt4'.format(col)].apply(len) > 0]

,user_id,spans_Anhedonia_expert,spans_Anhedonia_gpt4
0,45,[],[]
1,49,"[It’s becoming really stressful, we are not ha...",[not having answers of what is happening and w...
2,70,"[I am ok to around lunchtime, but then there i...",[absolutely nothing to do for the rest of the ...
3,71,"[Life feels like it is improving, I am able to...",[I am able to make future plans with excitement]
4,72,"[I've been stuck at home for the most part, I ...",[Mainly because I've been stuck at home for th...
...,...,...,...
204,573,[no motivation to complete even simple everyda...,[no motivation to get up and complete even sim...
205,575,[],[]
206,966,[not wanting to get out of bed],[not wanting to get out of bed]
207,499,[over the last few weeks I've just felt kind o...,[felt kind of numb]


In [43]:
print (temp.spans_Suicidal_ideation_expert.iloc[0] == [])

True
